[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ome/EMBL-EBI-imaging-course-04-2025/blob/main/ImageFormat.ipynb)
## Learning Objectives
* How to read various image formats

The reading functions of most Python library will return by default a [Numpy array](https://numpy.org/doc/stable/reference/generated/numpy.array.html). 

Note: **This cell will have to be copied to work inside the VM or on your own computer command line only. If you work in Google Colab, just take this cell as for your information only.**

In order to be able to work with BioIO library, which does not work in Google Colab at the moment, you can use the provided VMs to create a venv according to the [Setup.md](Setup.md).

After that, inside the newly created venv, run

```
wget https://downloads.openmicroscopy.org/images/ND2/aryeh/MeOh_high_fluo_011.nd2 # to get the image locally
pip install bioio
pip install bioio-nd2

python #this will open a python cmd line

from bioio import BioImage
import bioio_nd2

img = BioImage("./MeOh_high_fluo_011.nd2", reader=bioio_nd2.Reader)

img.dims.order # Return the dimension order. Note: This information is not available when using scikit-image only.

img.dims.X
img.channel_names
img.physical_pixel_sizes.X 
img.shape
img.data

# Return 4D CZYX numpy array
data = img.get_image_data("CZYX", T=0)
data.shape

# Return 3D ZYX numpy array
data = img.get_image_data("ZYX", T=0, C=0)
data.shape

# Get the id of the current operating scene. In our case we only have one scene
img.current_scene

# Get a list valid scene ids
img.scenes

# Save the image as OME-TIFF
img.save("MeOh_high_fluo_011.nd2.ome.tiff")





### imageio

When working with images in Python a library to consider is [scikit-image](https://scikit-image.org/) library. This library is useful when working with TIFF images. But, supporting a format does not necessary mean than all images in that format will be correctly read.
Another library to explore is [imageio](https://imageio.readthedocs.io/en/stable/). Scipy has deprecated their image I/O functionality for [imageio](https://imageio.readthedocs.io/en/stable/user_guide/scipy.html).

The `imageio` library supports reading local data or remote data

In [ ]:
# Download a file in the Colab environment so we can test the local reading
!wget https://github.com/imageio/imageio-binaries/raw/master/images/bacterial_colony.tif

In [ ]:
path = "/content/bacterial_colony.tif"
import imageio.v3 as iio
img = iio.imread(path)

In [ ]:
print(img.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img[0, :, :])

In [ ]:
path = "https://downloads.openmicroscopy.org/images/TIFF/condensation/A1.pattern1.tif"

In [ ]:
img = iio.imread(path)
print(img.shape)

To understand the image e.g. what are the dimensions, it is important to be able to read the metadata.
This information might not always be available in the image file itself.

In [ ]:
metadata = iio.immeta(path)
print(metadata.get('Info'))

In [ ]:
from ipywidgets import *

def update(t=0, c=0):
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(img[t, c, :, :])
    plt.tight_layout()
    fig.canvas.flush_events()

interact(update, t=widgets.IntSlider(value=0, min=0, max=img.shape[0]-1, step=1, description="Select T", continuous_update=False),
        c=widgets.IntSlider(value=0, min=0, max=img.shape[1]-1, step=1, description="Select C", continuous_update=False))

### tifffile
[tifffile](https://github.com/cgohlke/tifffile) is the library for working with TIFF data. 

In [ ]:
!pip install -U tifffile[all]

In [ ]:
import tifffile as tff
image = tff.imread('/content/bacterial_colony.tif')

In [ ]:
plt.imshow(image[0, :, :])

### BioIO and n2

In order to analyse your image data, you need to be able to read the numerous proprietary file formats (PFFs). Bio-Formats is the most comprehensive tool in Java to read PFFs, but no such library exists in Python yet. 

Nonetheless the [BioIO](https://github.com/bioio-devs/bioio) is a more domain-specific Python library for accessing data. It wraps several libraries for microscopy vendor's formats (including ``nd2``, ``czi``) and has a Python wrapper around the reference Java library for reading many proprietary file formats [Bio-Formats](https://www.openmicroscopy.org/bio-formats/). This is obviously not a pure Python solution since it requires Java installed in order to run. The library also offers the ability to convert to OME-TIFF. Unfortunately the most recent version BioIO library does not currently run on Google Colab.

Let's look at the [nd2 reader](https://github.com/tlambert03/nd2). It offers similar features to the ones currently supported by [BioIO](https://github.com/bioio-devs/bioio).
[nd2 reader](https://github.com/tlambert03/nd2) uses [ome-types]((https://ome-types.readthedocs.io/en/latest/)) which is the most advanced library in Python for working with the OME-XML that is stored in the OME-TIFF.

In [ ]:
!wget https://downloads.openmicroscopy.org/images/ND2/karl/sample_image.nd2

In [ ]:
%pip install nd2

In [ ]:
import nd2

In [ ]:
with nd2.ND2File('/content/sample_image.nd2') as ndfile:
    dims = ndfile.sizes
    print(dims)
    metadata = ndfile.metadata
    numpy_array = ndfile.asarray()  # in-memory numpy array
    array = ndfile.to_dask()

Exercise:
Modify the ``update`` method above to display the ``nd2`` image.

In [ ]:
%%time
numpy_array[0, 0, :, :]

In [ ]:
%%time
array[0, 0, :, :]